In [ ]:
import numpy as np
import open3d as o3d
from PIL import Image
import matplotlib.pyplot as plt
from utils.vis_utils import show_mask_ins, show_points_color

In [ ]:
device = "cuda:0"

# dataset params
dataset_dir = "/home/rpartsey/code/eai/SAMPro3D-fork/data/scannet/scans"
scene_name = "scene0000_02"

In [ ]:
# experiment params
experiments_dir = "/home/rpartsey/code/eai/SAMPro3D-fork/experiments/scannet/scans/3js_prompt"
experiment_name = "guitar"

prompt_path = f"{dataset_dir}/{scene_name}/{scene_name}_guitar.ply"
sam_output_path =  f"{experiments_dir}/{scene_name}/{experiment_name}/sam_output"
sampro3d_predictions =  f"{experiments_dir}/{scene_name}/{experiment_name}/sampro3d_predictions"
output_vis_path = f"{experiments_dir}/{scene_name}/{experiment_name}/visualization"

In [ ]:
i = 772
frame_path = f"{dataset_dir}/{scene_name}/color/{i}.jpg"
frame = Image.open(frame_path)
frame.show()


mask_path = f"{sam_output_path}/masks_npy/{i}.npy"
mask = np.load(mask_path)
print(mask.shape)

points_path = f"{sam_output_path}/points_npy/{i}.npy"
points = np.load(points_path)
print(points.shape)


def visualize_masks_and_points(frame, points):
    """
    Visualize masks and points on the given frame for specified mask indices.
    
    Args:
    frame (PIL.Image): The input image
    mask (np.array): Array of masks
    points (np.array): Array of points
    start_idx (int): Starting index of masks to visualize
    end_idx (int): Ending index of masks to visualize
    """
    # Convert PIL Image to numpy array
    frame_np = np.array(frame)
    
    # Create a figure and axis
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    
    # Display the frame
    ax.imshow(frame_np)
    
    # Plot masks and points
    show_points_color(points, np.ones(points.shape[0]), ax, np.random.rand(3), marker_size=200)
    
    ax.axis('off')
    plt.tight_layout()
    plt.show()

# Example usage:
start_idx = 0
end_idx = 2  # Visualize first 5 masks and points
visualize_masks_and_points(frame, points)